In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [4]:
path1 = "../SAEF/Working_Files/Reclassification_STRF/shapefiles/starterfiles"
bay_links = gpd.read_file(os.path.join(path1, "bayarealinks_clip.shp"))
len(bay_links) #650k links in Bay Area 9 counties

658662

## Step 1 - Transport Context for Links

In [47]:
# fc 1,2 , fc3 with speed greater than 80kph Highways , regex expy hwy # tried 90 previously , but ca92 in napa came as thoroghways
# Ramps - Ramps
# rest of Fc3, Fc4 Thouroughway
# Fc 5 - Neighborhood 
# capacity is not informative, so including all fc3 with speed >= 90 kph into highways

In [117]:
#ramps
ramps = bay_links[bay_links['RAMP']=='Y']['LINK_ID'].to_list()
print(len(ramps))

#highways
fc12speed60 = bay_links[(bay_links['FUNC_CLASS'].isin([1,2])) & (bay_links['SPEED_KPH']>= 60)]['LINK_ID'].to_list() #highways
print(len(fc12speed60))
fc3speed90 = bay_links[(bay_links['FUNC_CLASS']== 3) & (bay_links['SPEED_KPH']>= 90)]['LINK_ID'].to_list()
print(len(fc3speed90))

#all streets with hwy and expy belongs to highway category 
mask = bay_links[['ST_NAME']].apply(
    lambda x: x.str.contains(
        'HWY|EXPY|CA-',
        regex=True)).any(axis=1)
hwyexpy = bay_links[mask]['LINK_ID'].to_list()


fc12 = []
fc12.extend(fc12speed60)
fc12.extend(fc3speed90)
fc12.extend(hwyexpy)
print(len(fc12))
highways = set(fc12)- set(ramps) 
print("Num of Highways", len(highways))

10711
13235
4571
29161
Num of Highways 22379


In [118]:
#Thouroughways
fc12speedlower60 = bay_links[(bay_links['FUNC_CLASS'].isin([1,2])) & (bay_links['SPEED_KPH'] < 60)]['LINK_ID'].to_list()
print(len(fc12speedlower60))
fc34 = bay_links[bay_links['FUNC_CLASS'].isin([3, 4])]['LINK_ID'].to_list()
print(len(fc34))

thoroughways = set(fc34).union(set(fc12speedlower60))
thoroughways.difference_update(ramps)
thoroughways.difference_update(highways)
print("Thouroughways", len(thoroughways))

#neighborhood
neighborhood = set(bay_links[bay_links['FUNC_CLASS'].isin([5])]['LINK_ID'].to_list()) - set(ramps)-set(highways) # neighbourhood
print("Neighbourhood Streets", len(neighborhood))

577
148653
Thouroughways 130112
Neighbourhood Streets 495460


In [121]:
bay_links['transport_use'] = 0
bay_links.loc[bay_links['LINK_ID'].isin(ramps), ['transport_use']] = 'Highway Ramps'
bay_links.loc[bay_links['LINK_ID'].isin(highways), ['transport_use']] = 'Highway'
bay_links.loc[bay_links['LINK_ID'].isin(thoroughways), ['transport_use']] = 'Throughway'
bay_links.loc[bay_links['LINK_ID'].isin(neighborhood), ['transport_use']] = 'Neighborhood'

In [122]:
bay_links['transport_use'].value_counts()

Neighborhood     495460
Throughway       130112
Highway           22379
Highway Ramps     10711
Name: transport_use, dtype: int64

In [123]:
bay_links.groupby(['transport_use'])['LENGTH_met'].sum()/1000*0.621371 #miles

transport_use
Highway           2718.163521
Highway Ramps      802.856865
Neighborhood     38978.138402
Throughway        7626.536110
Name: LENGTH_met, dtype: float64

## Step 2 - Side Use Context

In [124]:
#Side use for links derived based on MTC landuse data using Arcgis spatial join - closest one
side_use1 = gpd.read_file(os.path.join(path1, "spatialjoin_landuse_links.shp")) # using the links file. Need to filetre bay arae links only 
len(side_use1)

1008959

In [125]:
side_use1 = side_use1[side_use1['LINK_ID'].isin(bay_links.LINK_ID)]
len(side_use1)

658662

In [ ]:
# Land use catgeories based on MTC data
type_id
0,Other,other/unknown,O,FALSE,FALSE,TRUE
1,Residential,single-family residential,HS,TRUE,FALSE,FALSE
2,Residential,town-home residential,HT,TRUE,FALSE,FALSE
3,Residential,multi-family residential,HM,TRUE,FALSE,FALSE
4,Office,office,OF,FALSE,TRUE,FALSE
5,Hotel,hotel,HO,FALSE,TRUE,FALSE
6,School,school,SC,FALSE,TRUE,FALSE
7,Industrial,light industrial,IL,FALSE,TRUE,FALSE
8,Industrial,warehouse,IW,FALSE,TRUE,FALSE
9,Industrial,heavy industrial,IH,FALSE,TRUE,FALSE
10,Retail,retail type 3,RS,FALSE,TRUE,FALSE
11,Retail,retail type 2,RB,FALSE,TRUE,FALSE
12,Residential,mixed-use with residential emphasis,MR,TRUE,TRUE,FALSE
13,Retail,retail type 1,MT,FALSE,TRUE,FALSE
14,Office,mixed-use with office emphasis,ME,TRUE,TRUE,FALSE
15,Parking,parking type 1,PA,FALSE,FALSE,TRUE
16,Parking,parking type 2,PA2,FALSE,FALSE,TRUE
99, Green/Open space

In [126]:
#aggregating the landuse for our classification
zoning_landuse = { 1:'Residential', 2:'Residential',3:'Residential',12:'Residential',
                 4:'Commercial',5:'Commercial', 10:'Commercial',11:'Commercial', 13:'Commercial',14:'Commercial',
                 6:'PSP',15:'PSP', 16:'PSP',
                 7:'Industrial', 8:'Industrial', 9:'Industrial',
                99:'Green/Open Space',
                0:'Other'}

In [127]:
side_use1['landuse'] = side_use1['type_id'].map(zoning_landuse)
side_use1['landuse'].value_counts()

Residential         407161
Other                89234
Commercial           69577
Green/Open Space     40225
PSP                  28310
Industrial           24155
Name: landuse, dtype: int64

In [128]:
side_use1['landuse'].isnull().sum()

0

In [129]:
side_use1.groupby(['landuse'])['LENGTH_met'].sum()/1000*0.621371 #miles

landuse
Commercial           3513.079534
Green/Open Space     2676.222931
Industrial           2027.290063
Other                8904.146268
PSP                  2805.671337
Residential         30199.284765
Name: LENGTH_met, dtype: float64

# Step 3 Combine transport and landuse context 

In [131]:
sc_classification = pd.merge(bay_links, side_use1[['LINK_ID','primary_id', 'developmen', 'land_value', 'acres',
       'county_id', 'zone_id', 'proportion', 'tax_exempt', 'apn', 'parcel_id_',
       'geom_id', 'imputation', 'x', 'y', 'shape_area', 'block_id', 'node_id',
       'parcel_id', 'developm_1', 'building_t', 'type_id', 'geometry',
       'landuse']], left_on = 'LINK_ID', right_on = 'LINK_ID', how = 'left')
len(sc_classification)

658662

In [132]:
sc_classification[(sc_classification['transport_use'] == 'Highway') & (sc_classification['landuse'] == 'Residential')]

,LINK_ID,ST_NAME,REF_IN_ID,NREF_IN_ID,FUNC_CLASS,DIR_TRAVEL,NUM_PHYS_L,SPEED_KPH,LENGTH_met,CAPACITY_v,...,y,shape_area,block_id,node_id,parcel_id,developm_1,building_t,type_id,geometry_y,landuse
741,23594242,CABRILLO HWY,48433001,48433002,5,F,1.0,40,15.152996,1000.0,...,37.539809,7.173942e+02,60816136001006,65521093,1130121.0,1.0,1.0,1.0,"LINESTRING (-122.51706 37.53957, -122.51689 37...",Residential
11319,23609862,GREAT HWY,48490231,48490234,4,F,1.0,40,60.564599,1000.0,...,37.735750,2.252366e+02,60750354005006,65320244,956295.0,1.0,1.0,1.0,"LINESTRING (-122.50611 37.73556, -122.50620 37...",Residential
15052,23615283,RICHARDSON AVE,48525065,48525001,2,F,3.0,60,63.532725,4500.0,...,37.797362,6.092960e+06,60750601001094,5427553440,914191.0,2.0,3.0,3.0,"LINESTRING (-122.44711 37.80036, -122.44765 37...",Residential
16843,23617944,CA-1,48436121,48436118,3,F,1.0,60,78.077232,1500.0,...,37.473276,1.216038e+03,60816135012065,65415466,1176263.0,1.0,1.0,1.0,"LINESTRING (-122.43529 37.47229, -122.43584 37...",Residential
16848,23617984,CA-1,48435390,48435386,3,F,1.0,80,77.754061,1500.0,...,37.495327,1.013814e+03,60816135022007,65488059,1155722.0,1.0,1.0,1.0,"LINESTRING (-122.45567 37.49521, -122.45646 37...",Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658628,1205968504,I-680,1148848294,1148966840,2,F,4.0,115,37.709896,8000.0,...,37.744752,6.682690e+02,60133451021005,57846181,689952.0,1.0,1.0,1.0,"LINESTRING (-121.95581 37.74459, -121.95561 37...",Residential
658629,1205968505,I-680,1148966841,1148848295,2,F,4.0,115,945.267107,8000.0,...,37.752851,1.158825e+03,60133451111000,57877880,727679.0,1.0,1.0,1.0,"LINESTRING (-121.96060 37.75245, -121.95594 37...",Residential
658630,1205968506,I-680,999960865,1148966841,2,F,4.0,115,82.553449,8000.0,...,37.753436,1.109601e+03,60133451111000,57830955,731489.0,1.0,1.0,1.0,"LINESTRING (-121.96103 37.75311, -121.96060 37...",Residential
658631,1205968974,I-680,1148967092,1147105795,2,F,5.0,115,466.200341,10000.0,...,37.791487,4.658588e+02,60133451051009,57867138,571088.0,1.0,1.0,1.0,"LINESTRING (-121.98318 37.79251, -121.98137 37...",Residential


In [133]:
sc_classification.groupby(['transport_use','landuse'])['LENGTH_met'].sum()/1000 #km

transport_use  landuse         
Highway        Commercial            481.275982
               Green/Open Space      454.852531
               Industrial            311.499598
               Other                1774.962925
               PSP                   345.379293
               Residential          1006.491177
Highway Ramps  Commercial            378.393109
               Green/Open Space      234.349940
               Industrial            106.135554
               Other                 305.497801
               PSP                    47.879628
               Residential           219.817247
Neighborhood   Commercial           2970.112001
               Green/Open Space     2408.610715
               Industrial           2165.265237
               Other                9541.907643
               PSP                  3025.946449
               Residential         42617.410531
Throughway     Commercial           1823.974128
               Green/Open Space     1209.151464
        

In [134]:
sc_classification.groupby(['transport_use','landuse'])['LENGTH_met'].sum()/1000 #km

transport_use  landuse         
Highway        Commercial            481.275982
               Green/Open Space      454.852531
               Industrial            311.499598
               Other                1774.962925
               PSP                   345.379293
               Residential          1006.491177
Highway Ramps  Commercial            378.393109
               Green/Open Space      234.349940
               Industrial            106.135554
               Other                 305.497801
               PSP                    47.879628
               Residential           219.817247
Neighborhood   Commercial           2970.112001
               Green/Open Space     2408.610715
               Industrial           2165.265237
               Other                9541.907643
               PSP                  3025.946449
               Residential         42617.410531
Throughway     Commercial           1823.974128
               Green/Open Space     1209.151464
        

In [135]:
# Final categorisation 
#combine ramps into highways for final categorisation 

def landuse_transp(df):
    if df['transport_use'] == 'Highway':
        return 'Highway'
    elif df['transport_use'] == 'Highway Ramps':
        return 'Highway'
    elif (df['transport_use'] == 'Throughway') & (df['landuse'] == 'Residential'):
        return 'Residential Throughway'
    elif (df['transport_use'] == 'Throughway') & (df['landuse'] == 'Commercial'):
        return 'Commercial Throughway'
    elif (df['transport_use'] == 'Throughway') & (df['landuse'] == 'Industrial'):
        return 'Industrial street'
    elif (df['transport_use'] == 'Throughway') & (df['landuse'] == 'PSP'):
        return 'PSP street'
    elif (df['transport_use'] == 'Neighborhood') & (df['landuse'] == 'Residential'):
        return 'Neighbourhood Residential street'
    elif (df['transport_use'] == 'Neighborhood') & (df['landuse'] == 'Commercial'):
        return 'Neighbourhood Commercial street'
    elif (df['transport_use'] == 'Neighborhood') & (df['landuse'] == 'Industrial'):
        return 'Industrial street'
    elif (df['transport_use'] == 'Neighborhood') & (df['landuse'] == 'PSP'):
        return 'PSP street'
    else:
         return 'Other/Openspace_street' 

In [137]:
sc_classification['new_classification'] = sc_classification.apply(lambda x: landuse_transp(x), axis = 1)
print(sc_classification['new_classification'].isnull().sum())
print(len(sc_classification))

0
658662


In [138]:
sc_classification.groupby(['new_classification'])['LENGTH_met'].sum()/1000*0.621371 #miles

new_classification
Commercial Throughway                1133.364628
Highway                              3521.020386
Industrial street                    1767.783691
Neighbourhood Commercial street      1845.541464
Neighbourhood Residential street    26481.222999
Other/Openspace_street               9859.380810
PSP street                           2561.311647
Residential Throughway               2956.069274
Name: LENGTH_met, dtype: float64

In [139]:
sc_classification.groupby(['new_classification'])['LENGTH_met'].sum()/sc_classification['LENGTH_met'].sum()*100 #percentage

new_classification
Commercial Throughway                2.261045
Highway                              7.024382
Industrial street                    3.526702
Neighbourhood Commercial street      3.681827
Neighbourhood Residential street    52.829638
Other/Openspace_street              19.669315
PSP street                           5.109778
Residential Throughway               5.897313
Name: LENGTH_met, dtype: float64

In [140]:
sc_classification.head(2)

,LINK_ID,ST_NAME,REF_IN_ID,NREF_IN_ID,FUNC_CLASS,DIR_TRAVEL,NUM_PHYS_L,SPEED_KPH,LENGTH_met,CAPACITY_v,...,shape_area,block_id,node_id,parcel_id,developm_1,building_t,type_id,geometry_y,landuse,new_classification
0,23592962,WHITMAN WAY,48501858,48501829,5,F,1.0,40,200.578658,1000.0,...,25020.541920,60816038021001,65432035,1086919.0,2.0,3.0,3.0,"LINESTRING (-122.42562 37.61791, -122.42776 37...",Residential,Neighbourhood Residential street
1,23592963,WHITMAN WAY,48501807,48501829,5,F,1.0,40,169.997630,1000.0,...,1152.061434,60816038021002,65407446,1087316.0,2.0,3.0,3.0,"LINESTRING (-122.42965 37.61779, -122.42776 37...",Residential,Neighbourhood Residential street


#### WRITE TO CSV AND SHAPEFILE

In [144]:
#writing files to local
writepath = "/Users/akuncheria/Documents/GSR-2021Feb/UCBerkeley_GSR/SAEF/Working_Files/Reclassification_STRF/final/may2022"
sc_classification.to_csv(os.path.join(writepath ,'bayarea_links_reclassification.csv'), index = False)

In [145]:
#write to shapefile 
bayclass = gpd.GeoDataFrame(sc_classification[['LINK_ID', 'ST_NAME', 'REF_IN_ID', 'NREF_IN_ID', 'FUNC_CLASS',
       'DIR_TRAVEL', 'NUM_PHYS_L', 'SPEED_KPH', 'LENGTH_met', 'CAPACITY_v',
       'RAMP', 'ref_lat', 'ref_long', 'nref_lat', 'nref_long',
       'transport_use', 'primary_id', 'developmen', 'land_value', 'acres',
       'county_id', 'zone_id', 'proportion', 'tax_exempt', 'apn', 'parcel_id_', 'imputation', 'x', 'y', 'shape_area', 'block_id', 'node_id',
       'parcel_id', 'developm_1', 'building_t', 'type_id',
       'landuse', 'new_classification']], crs = sc_classification.crs, geometry=sc_classification.geometry_x)


bayclass.to_file( os.path.join(writepath,"bayarea_links_reclassification.shp"), driver = 'ESRI Shapefile')